# Smoke

[![Google Collab Book](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tum-pbs/PhiFlow/blob/develop/examples/grids/Smoke_Plume.ipynb)

This example simulates a rising plume of hot smoke. It demonstrates how to advect and couple other quantities with the fluid.

In [2]:
# %pip install phiflow
from phi.jax.flow import *
from tqdm.notebook import trange
import numpy as np
# from phi.flow import *  # If JAX is not installed. You can use phi.torch or phi.tf as well.

We begin by defining our simulation size and smoke inflow.

In [4]:
domain = Box(x=32, y=32)
x = np.random.uniform(5, 27, size=4)  # Avoid boundaries
y = np.random.uniform(5, 27, size=4)
inflow = Sphere(x=x, y=y, radius=3)
inflow_rate = 0.2
print(inflow)

Sphere(vectorᶜ=2)


/opt/anaconda3/envs/mp-pde/lib/python3.11/site-packages/phiml/math/_shape.py:2144: RuntimeWarning: Stacking shapes with incompatible item names will result in item names being lost. For vector Got ('x', 'y') and None
  warnings.warn(f"Stacking shapes with incompatible item names will result in item names being lost. For {name} Got {item_names[index]} and {items}", RuntimeWarning)


Our simulation advects the velocity and smoke fields, computes the buoyancy force proportional to the smoke density, and enforces incompressibility.
We sample the velocities on a $64\times 64$ staggered grid, and the smoke density on a $200\times 200$ grid.

In [15]:
@jit_compile
def step(v, s, p, dt):
    s = advect.mac_cormack(s, v, dt) + inflow_rate * resample(inflow, to=s, soft=True)
    buoyancy = resample(s * (0, 0.1), to=v)
    v = advect.semi_lagrangian(v, v, dt) + buoyancy * dt
    v, p = fluid.make_incompressible(v, (), Solve('CG', 1e-3, x0=p))
    return v, s, p

v0 = StaggeredGrid(0, 0, domain, x=32, y=32)
smoke0 = CenteredGrid(0, ZERO_GRADIENT, domain, x=32, y=32)

In [16]:
v_trj, s_trj, p_trj = iterate(step, batch(time=100), v0, smoke0, None, dt=1, range=trange, substeps=3)

  0%|          | 0/100 [00:00<?, ?it/s]

In [22]:
s_trj

Grid[(timeᵇ=101, xˢ=32, yˢ=32) 0.878 ± 1.088 (0e+00...5e+00), ext=zero-gradient]

In [19]:
plot(s_trj, animate='time', frame_time=1000)